<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/lab_11_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_11_5: RNN-seqseq
## contents
- seq2seq
- apply seq2seq
  - encoder - decoder
  - data preprocessing
  - neural net setting
  - training
  - evaluation
---
## seq2seq
- sequnce 입력받아 sequence 출력하는 모델
- 이미 RNN이 sequential data를 다루기 위해 만들어졌다고 했는데 일반적인 RNN과 뭐가 다른가?

- Example: chatbot
  - RNN : "Today's perfect weather makes me much sad." 라는 문장을 보고 한 단어씩 추출하여 답변을 생성한다면, 위로랑은 거리가 먼 문장을 생성해낼 것.   
  끝에서 살짝 변하는 말에 대해 대응하지 못한다.
  - seq2seq : 문장 다 보기 전에 답변을 만들면 오류 발생 -> 끝까지 다 듣고 말하기 모델 

## apply seq2seq
### 1. encoder-decoder
- seq2seq의 대표적 특징: encoder - decoder 구조

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdwHwrS%2Fbtq0LO05pec%2Fj0r86KSbeRHuxkOZYDIPQ0%2Fimg.png" width="500px" height="300px"></img>
- encoder
  - 입력된 sequence를 ventor 형태로 압축하여 decoder에 전달
  - 모든 문장을 들은 후
- decoder
  - cell의 output과 hidden state를 이용해서 단어 예측
  1. 첫 cell에서 encoder로부터 전달받은 vector를 hiddenstate로 넣어주고 start flag와 함께 모델 시작
  2. 첫 cell의 output을 답변 문장의 첫 단어로 사용
  3. output은 두 번째 cell의 입력으로, hidden state와 함께 다음 단어 예측
  4. 예측이 진행되면 decoder에서 하나의 문장(답변)을 만들어냄


- 영한 번역 code
  - source text(영어) -> target text(한국어)

In [ ]:
import random
import torch
import torch.nn as nn
from torch import optim

#대략
# declare max length for sentence
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH) # Preprocess: source, target으로 나누고 train, test data으로 나눈다.
print(random.choice(load_pairs))

# declare the encoder and the decoder
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

# train seq2seq model
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)# Encoder 출력을 Decoder에 넣는 코드를 포함하고 있을 것이다.

# check the model with given data
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

### 2. data preprocessing


In [ ]:
import random
import torch
import torch.nn as nn
from torch import optim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 영어 tab 한국어 
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]
       
# token 
# decoder의 마지막 출력물을 받아오고 첫 번째 step의 index를 받아온다.
SOS_token = 0 # "start of sentence" step의 input
EOS_token = 1 # "end of sentence" 문장의 종료 알림

In [ ]:
# preprocessing: Source text와 target text를 나눠서 어떤 단어로 구성, 단어의 수 등을 특정하는 부분
# data 준비 과정
def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])# tab으로 구분
    print("Read {} sentence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs".format(len(pairs)))

    # Vocab() class를 선언하여 단어 대수나 dictionary 만들어서 넣음
    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

### 3. neural net setting
- encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        # embedding은 거대한 행렬
        # input_size에서 hidden size 만큼의 vector로 줄인다.
        self.embedding = nn.Embedding(input_size, hidden_size) # input_size: Source에서 쓰이는 단어의 개수

        # EMB(가로로 긴 vector) * one-hot encoding(세로로 긴) -> hidden size로 줄어든 vector -> 이를 gru에서 처리한다.       
        
        self.gru = nn.GRU(hidden_size, hidden_size) # 들어오는 dim과 나오는 dim(hidden state)만 정해주면 된다.

    # encoder구조를 알 수 있다.
    def forward(self, x, hidden):
        # embedding을 통과해서 나오고 , 이것이 gru에 input으로 들어가서 나온다.
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

- decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size) # target에 맞추어 그만큼 복
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden
        # one hot encoding * EMB -> 줄어든 vector -> GRU(decoder) <- hidden state(dim=n)
        # ->(dim=n) -> out > target text에서 쓰이는 단어로 복원

### 4. training

In [ ]:
def tensorize(vocab, sentence): # 학습 전 sentence를 입력 받아서 onehot encoding을 하고 이를 tensor로 바꿔주는 역할 보조함수
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1)
    
# training seq2seq
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    # category vector를 비교할 때 많이 사용되는 loss이다.
    # Negative Log Likelihood Loss
    # 마지막에 nn.LogSoftmax 를 해주어야한다.
    criterion = nn.NLLLoss()

    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]

        # hidden state
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device) # 첫 hidden state: 0벡터 [0 0 0]

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        
        for enc_input in range(source_length):
            # 문장 끝날 때까지 루프 돌면서 hiddeen state를 꺼냄.
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        # encoder의 마지막 hidden state를 decoder의 첫 hidden state로. input으로 Start_Of_Token(sos)
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden # connect encoder output to decoder input

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing: 실제 값을 넣어줘서 훈련하는 방법이 있다. 빠르지만, 네트워크 학습이 불안정할 수 있다.

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))